## HW2:Сегментируй и властвуй

### Цель
Чтобы попрактиковаться в изученных методах вам предлагается взять одну из задачек на выбор (а если очень хочется, то и все две) - кластеризация финансовых временных рядов или работа с биосигналами человека, для нахождения сегментов с одинаковым поведением

### Описание задачи
EEG считывает электрические сигналы мозга и позволяет понять, насколько активен мозг в данный момент. Приступ эпилепсии характеризуется аномальной активностью мозга, которую можно попытаться задетектировать при помощи EEG, что мы и будем делать в рамках задачи.

* Скачайте данные из UCI репозитория - https://huggingface.co/datasets/ankislyakov/EpilepticSeizureRecognition/resolve/main/Epileptic%20Seizure%20Recognition.csv.
* Проведите EDA, а также трансформируйте целевую переменную из многоклассовой в бинарную - класс 1 (приступ) против всех остальных и отложите 20% данных для тестирования (не забудьте про стратификацию).
* Постройте бейзлайн, используя сырые данные в качестве признаков, без дополнительной предобработки. В качестве модели (или даже моделей) можете выбрать любой полюбившийся вам алгоритм.
* Так как каждый временной ряд представляет собой сигнал из 178 наблюдений, пришедших за 1 секунду, имеет смысл попробовать методы извлечения признаков из сигналов. Извлеките FFT и Wavelet признаки и снова обучите модель/модели и проверьте качество.
* Наконец, при помощи библиотеки для автоматического извлечения признаков из временных рядов сгенерируйте статистические признаки и снова проверьте качество моделей.
* Какой из подходов показал лучшее качество?

### Библиотеки

In [1]:
!pip install PyWavelets

In [2]:
!pip install tsfresh

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_auc_score,
                             accuracy_score,
                             f1_score,
                             precision_recall_curve,
                             confusion_matrix,
                             PrecisionRecallDisplay,
                             ConfusionMatrixDisplay,
                             classification_report)
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

from scipy.fft import fft
import pywt

from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import select_features

In [4]:
# Константы
RANDOM_SEED = 42

### Загрузка и предобработка данных

In [5]:
data = pd.read_csv("https://huggingface.co/datasets/ankislyakov/EpilepticSeizureRecognition/resolve/main/Epileptic%20Seizure%20Recognition.csv")

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11500 entries, 0 to 11499
Columns: 180 entries, Unnamed to y
dtypes: int64(179), object(1)
memory usage: 15.8+ MB


In [7]:
data.shape

(11500, 180)

В датасете 180 колонок, смотреть, что то в них глазами не имеет смысла.

#### Поиск пропусков

In [8]:
def get_missing_values(dataframe):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)

    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)

    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])

    print(missing_df, end="\n")

In [9]:
get_missing_values(data)

Empty DataFrame
Columns: [n_miss, ratio]
Index: []


Пропусков в датасете не обнаружено.

### Подготовка данных к обучению


In [10]:
data['y'].value_counts()

,count
y,
4,2300
1,2300
5,2300
2,2300
3,2300


По условию задачи выделим все значения кроме `1` в отдельный класс `0`.

In [11]:
data['y'] = data['y'].apply(lambda x: 1 if x < 2 else 0)

In [12]:
data['y'].value_counts()

,count
y,
0,9200
1,2300


In [13]:
data.head()

,Unnamed,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [14]:
data = data.drop('Unnamed', axis=1)

Осталось два класса.

In [15]:
def prepare_data(X, y, test_size):
  X_train, X_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      test_size=test_size,
                                                      random_state=RANDOM_SEED,
                                                      stratify=y)

  scaler = StandardScaler()

  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  return X_train, X_test, y_train, y_test

In [16]:
X  = data.drop(['y'], axis=1)
y = data['y']

In [17]:
X_train, X_test, y_train, y_test = prepare_data(X, y, 0.3)

### Logistic Regression
Сначала обучим простую модель логической регресси буквально в лоб, без какой либо работы с признаками.

In [18]:
def train_logistic_regression(X_train, X_test, y_train, y_test):
    model = LogisticRegression()
    model = model.fit(X_train, y_train)
    pred = model.predict(X_test)

    print(classification_report(y_test, pred));

In [19]:
train_logistic_regression(X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      2760
           1       0.97      0.10      0.19       690

    accuracy                           0.82      3450
   macro avg       0.89      0.55      0.54      3450
weighted avg       0.85      0.82      0.76      3450



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### FTT

In [20]:
def extract_fft_features(signal, sampling_rate=178, n_features=10):
    fft_result = fft(signal)
    magnitude = np.abs(fft_result)  # Амплитуда
    freq = np.fft.fftfreq(len(signal), 1/sampling_rate)

    # Возьмем первые n_features частот (игнорируем отрицательные частоты)
    positive_freq = freq[:len(freq)//2]
    positive_magnitude = magnitude[:len(magnitude)//2]

    # Выбираем топ-N частот с наибольшей амплитудой
    top_freq_indices = np.argsort(positive_magnitude)[-n_features:]
    top_freqs = positive_freq[top_freq_indices]
    top_magnitudes = positive_magnitude[top_freq_indices]

    return np.concatenate([top_freqs, top_magnitudes])

In [21]:
columns_n = X_train.shape[1]
columns_n

178

In [22]:
X_train_fft = np.array([extract_fft_features(x, sampling_rate=columns_n) for x in X_train])
X_test_fft = np.array([extract_fft_features(x, sampling_rate=columns_n) for x in X_test])

In [23]:
train_logistic_regression(X_train_fft, X_test_fft, y_train, y_test)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2760
           1       0.92      0.84      0.88       690

    accuracy                           0.95      3450
   macro avg       0.94      0.91      0.92      3450
weighted avg       0.95      0.95      0.95      3450



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Wavelet


In [24]:
def extract_wavelet_features(signal, wavelet='db4', max_level=3):
    coeffs = pywt.wavedec(signal, wavelet, level=max_level)
    features = []
    for coeff in coeffs:
        features.extend([
            np.mean(coeff), np.std(coeff), np.min(coeff), np.max(coeff)
        ])
    return features

In [25]:
X_train_wavelet = np.array([extract_wavelet_features(x) for x in X_train])
X_test_wavelet = np.array([extract_wavelet_features(x) for x in X_test])

In [26]:
train_logistic_regression(X_train_wavelet, X_test_wavelet, y_train, y_test)

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      2760
           1       0.95      0.84      0.89       690

    accuracy                           0.96      3450
   macro avg       0.96      0.92      0.94      3450
weighted avg       0.96      0.96      0.96      3450



### tsfresh

In [27]:
# Преобразуем X_train и X_test в формат для tsfresh
def prepare_ts_data(X):
    ts_data = []
    for i, signal in enumerate(X):
        for t, value in enumerate(signal):
            ts_data.append({'id': i, 'time': t, 'value': value})
    return pd.DataFrame(ts_data)

# Создаем DataFrame для train и test
X_train_ts = prepare_ts_data(X_train)
X_test_ts = prepare_ts_data(X_test)

In [28]:
X_train_ts.head()

,id,time,value
0,0,0,-0.247496
1,0,1,-0.288509
2,0,2,-0.220710
3,0,3,-0.227322
4,0,4,0.047497


In [29]:
train_features = extract_features(
    X_train_ts,
    column_id='id',
    column_sort='time',
    default_fc_parameters=MinimalFCParameters(),
    n_jobs=4
)

Feature Extraction: 100%|██████████| 20/20 [00:05<00:00,  3.95it/s]


In [30]:
test_features = extract_features(
    X_test_ts,
    column_id='id',
    column_sort='time',
    default_fc_parameters=MinimalFCParameters(),
    n_jobs=4
)

Feature Extraction: 100%|██████████| 20/20 [00:03<00:00,  5.02it/s]


In [31]:
train_features.head()

,value__sum_values,value__median,value__mean,value__length,value__standard_deviation,value__variance,value__root_mean_square,value__maximum,value__absolute_maximum,value__minimum
0,25.168542,0.134828,0.141396,178.0,0.341107,0.116354,0.369252,1.083995,1.083995,-0.590799
1,-43.787775,-0.224005,-0.245999,178.0,0.303845,0.092322,0.390944,0.239571,1.294134,-1.294134
2,-56.088896,-0.298728,-0.315106,178.0,0.209886,0.044052,0.378608,0.078320,0.960420,-0.960420
3,32.127648,0.336078,0.180492,178.0,0.762818,0.581891,0.783881,1.576140,2.071844,-2.071844
4,16.782139,0.072028,0.094282,178.0,1.355872,1.838390,1.359146,2.997767,2.997767,-2.724405


In [32]:
impute(train_features)
impute(test_features)
y_train_reset = y_train.reset_index(drop=True)

train_features_filtered = select_features(train_features, y_train_reset)
test_features_filtered = test_features[train_features_filtered.columns]

In [33]:
train_logistic_regression(train_features_filtered, test_features_filtered, y_train, y_test)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2760
           1       0.93      0.85      0.89       690

    accuracy                           0.96      3450
   macro avg       0.95      0.92      0.93      3450
weighted avg       0.96      0.96      0.96      3450



### Результаты

In [34]:
models = ["LR", "FFT", "Wivelet", "tsfresh"]
acc = [0.82, 0.95, 0.96, 0.96]

df = pd.DataFrame({
    'Модели': models,
    'accuracy': acc
})

df

,Модели,accuracy
0,LR,0.82
1,FFT,0.95
2,Wivelet,0.96
3,tsfresh,0.96


Все три метода работы с временными рядами - FFT, Wivelet и tsfresh показали высокие результаты. И судя по метриками на данном наборе данных разницы между ними практически нет.